# Program Evaluation and Review Technique (PERT)

## Introduction to optimization and operations research.

Michel Bierlaire


In [ ]:

from typing import NamedTuple

import pandas as pd
from IPython.core.display_functions import display
from matplotlib import pyplot as plt
from networkx import DiGraph
from teaching_optimization.networks import draw_network
from teaching_optimization.networks.shortest_path_algorithm import ShortestPathAlgorithm



The renovation of an apartment's living room breaks down into several
tasks listed below. Precedence constraints have
to be respected during the planning of the works. The duration of each task is given as well.

The objective of this exercise is to

- identify critical tasks that cannot suffer from any delay without delaying the project,
- give the minimal timing of the duration of the work,
- calculate the earliest starting dates of each task,
- calculate the latest starting dates of each task.

We first define a data structure for the tasks.

In [ ]:
class Task(NamedTuple):
    name: str
    description: str
    precedence: list[str]
    duration: float



Here is the list of tasks with the precedence constraints, and the duration.

In [ ]:
tasks_list: list[Task] = [
    Task('A', 'Door removal', [], 0.5),
    Task('B', 'Sanding and painting doors', ['A'], 3),
    Task('C', 'Hanging doors', ['B', 'J'], 0.5),
    Task('D', 'Peeling off wallpapers', [], 1),
    Task('E', 'Pulling electrical wires', ['D'], 1),
    Task('F', 'Laying electrical outlet', ['E', 'H', 'I'], 0.5),
    Task('G', 'Smoothing walls', ['E', 'A'], 2),
    Task('H', 'Sanding of frames', ['G'], 2),
    Task('I', 'Ceiling painting', ['G'], 3),
    Task('J', 'Painting frames', ['H', 'I'], 1),
    Task('K', 'Ripping off the carpet', ['H', 'I', 'J'], 0.5),
    Task('L', 'Sanding parquet', ['K'], 1),
    Task('M', 'Impregnation and drying of parquet', ['L', 'F'], 4),
    Task('N', 'Balcony painting', [], 2),
    Task('O', 'Change of solar protections', ['N'], 1),
]



Printing the list of tasks.

In [ ]:
for task in tasks_list:
    print(task)


# Network representation

We first need to provide a the_network representation of the problem.

We create a the_network where each node is a task.
We also add one node "b" to represent the beginning of the project, and one node "e" to represent the end of the
project. We also associate each node with a coordinate in order to display the the_network.

In [ ]:

positions = {
    'b': (0, 0),
    'A': (1, 2),
    'B': (2, 2),
    'C': (6, 2),
    'D': (1, 0),
    'E': (2, 0),
    'F': (5, -1),
    'G': (3, 0),
    'H': (4, 1),
    'I': (4, 0),
    'J': (5, 1.5),
    'K': (6, 0),
    'L': (7, 0),
    'M': (8, 0),
    'N': (1, -2),
    'O': (5, -2),
    'e': (9, 0),
}

nodes = list(positions.keys())


Now, generate the list of arcs of the the_network from the task description. Each arc is a tuple
``(upstream_node, downstream_node, duration)``

First, we create a dict with the durations of the tasks.

In [ ]:
tasks_duration: dict[str, float] = {task.name: task.duration for task in tasks_list}


For each task $j$, we add an arc $(i,j)$ for each predecessor $i$.

In [ ]:
list_1 = [
    (predecessor, task.name, tasks_duration[predecessor])
    for task in tasks_list
    for predecessor in task.precedence
]
display(list_1)


For each task $j$ without predecessor, we define an arc $(b,j)$.

In [ ]:
list_2 = [
    ('b', task.name, 0.0) for task in tasks_list if not task.precedence
]
display(list_2)


For each task $i$ without successor, we define an arc $(i,e)$.

First, we identify the set of tasks without successor.

In [ ]:
all_tasks = {task.name for task in tasks_list}
task_with_successor = {
    predecessor for task in tasks_list for predecessor in task.precedence
}
tasks_without_successor = all_tasks - task_with_successor
display(tasks_without_successor)


Then, we create the last list of arcs.

In [ ]:
list_3 = [
    (task, 'e', tasks_duration[task]) for task in tasks_without_successor
]
display(list_3)


We merge the three lists

In [ ]:
arcs = list_1 + list_2 + list_3
display(arcs)


We create and display the the_network

In [ ]:
pert_network: DiGraph = DiGraph()
for node in nodes:
    pert_network.add_node(node, pos=positions[node])
pert_network.add_weighted_edges_from(arcs, weight='cost')


In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
draw_network(the_network=pert_network, attr_edge_labels='cost', ax=ax)
plt.show()


# Identification of the critical path

We need to calculate the longest path from $b$ to $e$. To do that, we need
to calculate the shortest path in a the_network where the cost is the negative of the duration.

Change the sign of all the weights on the arcs

In [ ]:
for _, _, data in pert_network.edges(data=True):
    data['cost'] = -data['cost']


We draw again the the_network with the new costs.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
draw_network(the_network=pert_network, attr_edge_labels='cost', ax=ax)
plt.show()


We now calculate the shortest paths.

Initialization

In [ ]:
the_algorithm = ShortestPathAlgorithm(
    the_network=pert_network,
    the_cost_name='cost',
    the_origin='b',
)


Running the algorithm

In [ ]:
the_algorithm.shortest_path_algorithm()
display(the_algorithm.iterations)


Printing the shortest path from 'b' to 'e'.

Each node along that path corresponds to a critical task.

In [ ]:
print(the_algorithm.recursive_shortest_path(node='e'))


The minimal timing of the duration of the work.

Remember that the optimal labels are available at ``the_algorithm.labels``

It is given by the label of node 'e', with the opposite sign.

In [ ]:
minimal_timing = -the_algorithm.labels['e']
print(f'Minimal timing for the project: {minimal_timing} days')


# Earliest starting dates of each task
The optimal labels of the algorithm are the earliest start for each task, with the opposite sign.

In [ ]:
earliest_starting_day = {
    task.name: -the_algorithm.labels[task.name]
    for task in tasks_list
}
print(f'Earliest starting day:')
for task, earliest in earliest_starting_day.items():
    print(f'Task {task}: {earliest} days.')


# Latest starting dates of each task

In order to obtain the latest starting day for each task, we have to look at the problem from the end, and  not from
the beginning. In order to start from the end and work backward, we need to
invert the direction for each arc, and then calculate again the longest
path, but from $e$ to $b$. All labels are then be interpreted as a number of days until the end of the project,
instead of a number of days since the beginning.

In [ ]:

pert_reversed = DiGraph()
pert_reversed.add_nodes_from(
    pert_network.nodes(data=True)
)  # Copy nodes with attributes
for upstream, downstream, data in pert_network.edges(data=True):
    pert_reversed.add_edge(
        downstream, upstream, cost=data['cost']
    )  # Reverse the direction of the arc


We draw the updated the_network

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
draw_network(the_network=pert_reversed, attr_edge_labels='cost', ax=ax)
plt.show()


We solve the shortest path problem in the new the_network, starting from the end of the project.

Initialization

In [ ]:
the_reversed_algorithm = ShortestPathAlgorithm(
    the_network=pert_reversed,
    the_cost_name='cost',
    the_origin='e',
)


Running the algorithm

In [ ]:
the_reversed_algorithm.shortest_path_algorithm()
display(the_reversed_algorithm.iterations)


Printing the shortest path from 'e' to 'b'.
This path involves the same nodes as before, corresponding to the critical tasks.

In [ ]:
print(the_reversed_algorithm.recursive_shortest_path(node='b'))


The minimal timing of the duration of the work.

It is given by the label of node 'b', with the opposite sign.
It must be the same as before.

In [ ]:
minimal_timing = -the_reversed_algorithm.labels['b']
print(f'Minimal timing for the project: {minimal_timing} days')


The (opposite of the) optimal labels of the "reversed" algorithm give us how many days before the end of the project should
each task start at the latest.
Consider task $O$ for instance. It does not precede
any task, and lasts 1 day. Therefore, it can start at the latest one day before the end of the project,
that is at day $13.5-1=12.5$, without delaying the project.

Therefore, to obtain the
latest start for each task, we just need to deduce this quantity from the duration of the project.

In [ ]:

latest_starting_day = {
    task.name: minimal_timing
    - (-the_reversed_algorithm.labels[task.name])
    for task in tasks_list
}
print(f'Latest starting days')
print(latest_starting_day)



# Summary
For each task, we report the earliest and latest starting day. If they are equal, it is a
critical task.

In [ ]:
summary = []
for task in tasks_list:
    is_critical = earliest_starting_day[task.name] == latest_starting_day[task.name]
    row = {
        'Name': task.name,
        'Description': task.description,
        'Earliest start': earliest_starting_day[task.name],
        'Latest start': latest_starting_day[task.name],
        'Critical': 'YES' if is_critical else 'no',
    }
    summary.append(row)
display(pd.DataFrame(summary))